In [151]:
import numpy as np

problem = """
--- Day 3: Gear Assembly ---
The missing part wasn't the only issue - one of the gears in the engine is wrong. A gear is any * symbol that is adjacent to exactly two part numbers. Its gear ratio is the result of multiplying those two numbers together.

This time, you need to find the gear ratio of every gear and add them all up so that the engineer can figure out which gear needs to be replaced.

Consider the same engine schematic again:

In this schematic, there are two gears. The first is in the top left; it has part numbers 467 and 35, so its gear ratio is 16345. The second gear is in the lower right; its gear ratio is 451490. (The * adjacent to 617 is not a gear because it is only adjacent to one part number.) Adding up all of the gear ratios produces 467835.

What is the sum of all of the gear ratios in your engine schematic?
"""

example_input = """
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
"""
real_input = True
# Read the rucksack contents from the file
if real_input:
    with open("adventofcode.com_2023_day_3_input.txt", "r") as infile:
        example_input = infile.read()
# replace anything not number or '*' with '.'
# example_input = "".join([x if x.isdigit() or x == '*' or x == '\n' else '.' for x in example_input])
input_content = [list(line.strip()) for line in example_input.strip().split("\n")]
# input_content = input_content[26:30]

def extract_number(content_array, pos):
    # Start at the given position
    left, right = pos[1], pos[1]

    # Scan left until a non-numeric character is found or the array boundary is hit
    while left > 0 and content_array[pos[0], left-1].isdigit():
        left -= 1

    # Scan right until a non-numeric character is found or the array boundary is hit
    while right < len(content_array[0])-1 and content_array[pos[0], right+1].isdigit():
        right += 1

    # Extract the full number as a string
    number_str = content_array[pos[0], left:right+1]

    # Convert the number to an integer and return it
    return int(''.join(number_str))


def find_parts(content_array):
    gears = []
    # Print the index of all '*'s
    gear_locations = np.where(content_array == '*')
    gear_locations = list(zip(gear_locations[0], gear_locations[1]))
    # print(gear_locations)
    # Check +/-1 each position and see if there is a number
    for pos in gear_locations:
        print(f"Checking position {pos}")
        # Check if any immediate neighbours are numbers
        parts = []
        for i in range(-1, 2):
            if pos[0] + i < 0:
                continue
            # print(f"Scanning Row {content_array[pos[0]+i, :]}")
            for j in range(-1, 2):
                if pos[1] + j < 0:
                    continue
                # print(f"Scanning column {content_array[:, pos[1]+j]}")
                # Guard against overbounds
                try:
                    if content_array[pos[0] + i, pos[1] + j].isdigit():
                        # print(f"Found a number at {pos[0] + i, pos[1] + j}, {content_array[pos[0] + i, pos[1] + j]}")
                        parts.append([pos[0] + i, pos[1] + j])
                except IndexError:
                    pass
        # Eliminate any parts right next to each other on same row only
        parts_copy = parts.copy()
        for part in parts_copy:
            part_row = part[0]
            if [part_row, part[1] + 1] in parts or [part_row, part[1] - 1] in parts:
                parts.remove(part)
                print(f"Removed {part} from {parts}")
        gears.append([pos, parts])
    return gears

def calculate_ratio(gear_parts):
    total_gear_ratio = 0
    for candidate in gear_parts:
        if len(candidate[1])==2:
            print(f"\nFound a candidate at {candidate[0]} with part locations{candidate[1]}")

            gear_ratio = 1
            # Search for the full number in the row
            for part_pos in candidate[1]:
                row_content = content_array[part_pos[0], :]
                # split on '.'
                min_index = part_pos[1] - 3 if part_pos[1] - 3 >= 0 else 0
                max_index = part_pos[1] + 3 if part_pos[1] + 3 <= len(row_content) else len(row_content)
                # Replace all not isalnum with '.'
                row_content = "".join([x if x.isalnum() else '.' for x in row_content])
                row_content = "".join(row_content[min_index:max_index]).split('.')
                # trim all special characters for all strings in the list
                # print(f"Found row content {row_content}")
                num = extract_number(content_array, part_pos)
                gear_ratio *= num
            total_gear_ratio += gear_ratio
            print(f"Found a gear ratio of {gear_ratio}, numbers were {gear_ratio//num}, {num}")
        else:
            print(f"\nFound a candidate at {candidate[0]} with part locations{candidate[1]} but wrong number of parts")
        # Display 3x3 grid around the candidate
        min_row = candidate[0][0] - 2 if candidate[0][0] - 2 >= 0 else 0
        max_row = candidate[0][0] + 2 if candidate[0][0] + 2 <= len(content_array) else len(content_array)
        min_col = candidate[0][1] - 3 if candidate[0][1] - 3 >= 0 else 0
        max_col = candidate[0][1] + 4 if candidate[0][1] + 4 <= len(content_array[0]) else len(content_array[0])
        print(content_array[min_row:max_row, min_col:max_col])
    return total_gear_ratio

# Find the index of 
content_array = np.array(input_content)
print(content_array)
print(content_array.shape)
# Print the index of all '*'s
gear_parts = find_parts(content_array)
print(gear_parts)
print(calculate_ratio(gear_parts))


# Found a candidate at (19, 63) with part locations[[18, 63], [20, 62], [20, 64]] but wrong number of parts
# [['.' '.' '.' '.' '.' '.' '.']
#  ['.' '.' '1' '1' '1' '.' '.']
#  ['.' '.' '.' '*' '.' '.' '.']
#  ['.' '.' '3' '0' '5' '.' '.']]

# Checking position (19, 63)
# Removed [18, 62] from [[18, 63], [18, 64], [20, 62], [20, 63], [20, 64]]
# Removed [18, 64] from [[18, 63], [20, 62], [20, 63], [20, 64]]
# Removed [20, 63] from [[18, 63], [20, 62], [20, 64]]

[['.' '.' '.' ... '.' '.' '.']
 ['.' '.' '.' ... '.' '.' '.']
 ['.' '.' '.' ... '.' '.' '.']
 ...
 ['.' '.' '.' ... '.' '.' '.']
 ['.' '.' '.' ... '.' '.' '.']
 ['.' '.' '.' ... '.' '.' '.']]
(140, 140)
Checking position (1, 72)
Removed [0, 71] from [[0, 72], [2, 71]]
Checking position (1, 119)
Checking position (2, 34)
Checking position (2, 45)
Removed [1, 45] from [[1, 46], [3, 46]]
Checking position (2, 113)
Removed [1, 112] from [[1, 113], [3, 114]]
Checking position (3, 7)
Checking position (3, 63)
Removed [2, 62] from [[2, 63], [4, 62], [4, 63]]
Removed [4, 62] from [[2, 63], [4, 63]]
Checking position (3, 109)
Removed [2, 108] from [[2, 109], [4, 110]]
Checking position (3, 131)
Removed [2, 130] from [[2, 131]]
Checking position (3, 133)
Checking position (4, 29)
Checking position (4, 72)
Removed [5, 72] from [[3, 73], [5, 73]]
Checking position (4, 102)
Checking position (5, 37)
Checking position (5, 129)
Removed [6, 129] from [[4, 128], [6, 130]]
Checking position (6, 89)
Chec

In [ ]:
# Errors
# Found a gear ratio of 5576, numbers were 164, 34
# Found a gear ratio of 8946, numbers were 9, 994
# Found a gear ratio of 46116, numbers were 854, 54


# Expected
# Found a gear ratio of 1148, numbers were 164, 7
# Found a gear ratio of 2982, numbers were 3, 994
# Found a gear ratio of 5124, numbers were 854, 6



# Found a candidate at (55, 120) with part locations[[54, 121], [56, 120]]
# Found a gear ratio of 5576, numbers were 164, 34
# [['.' '.' '.' '.' '.' '.' '.']
#  ['.' '.' '.' '1' '6' '4' '.']
#  ['.' '$' '.' '*' '.' '.' '.']
#  ['3' '4' '.' '7' '.' '.' '.']]

# Found a candidate at (46, 132) with part locations[[46, 131], [46, 133]]
# Found a gear ratio of 8946, numbers were 9, 994
# [['2' '8' '.' '.' '.' '.' '.']
#  ['.' '.' '.' '.' '.' '.' '.']
#  ['.' '.' '3' '*' '9' '9' '4']
#  ['*' '.' '.' '.' '.' '.' '.']]

# Found a candidate at (42, 3) with part locations[[41, 2], [41, 4]]
# Found a gear ratio of 46116, numbers were 854, 54
# [['.' '.' '.' '.' '.' '.' '@']
#  ['8' '5' '4' '.' '6' '.' '.']
#  ['.' '.' '.' '*' '.' '.' '.']
#  ['.' '.' '.' '.' '.' '.' '.']]

In [ ]:
def file_to_list(filename: str) -> list:
    """
    Read the file and return a list of the strings of each line.
    :param filename:
    :return content:
    """
    with open(filename) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    return content

def list_of_strings_to_list_of_lists(list_of_strings: list) -> list:
    """
    Convert a list of strings to a list of lists.
    :param list_of_strings:
    :return list_of_lists:
    """
    list_of_lists = []
    for string in list_of_strings:
        list_of_lists.append([x for x in string])
    return list_of_lists

def is_adjacent_to_symbol(list_of_lists, row, col) -> (bool, tuple):
    """
    Check if the character at the given row and column is adjacent to a symbol.
    Get the coordinates of the adjacent symbol.
    :param list_of_lists:
    :param row:
    :param col:
    :return:
    """
    # Directions to check: (row_change, col_change)
    directions = [(-1, -1), (-1, 0), (-1, 1),  # Up Left, Up, Up Right
                  (0, -1), (0, 1),  # Left, Right
                  (1, -1), (1, 0), (1, 1)]  # Down Left, Down, Down Right

    for dr, dc in directions:
        new_row, new_col = row + dr, col + dc

        # Check if new_row and new_col are within the bounds of the list
        if 0 <= new_row < len(list_of_lists) and 0 <= new_col < len(list_of_lists[0]):
            adjacent_char = list_of_lists[new_row][new_col]
            # Check if the adjacent character is a '*'
            if adjacent_char == "*":
                # Return True and the coordinates of the adjacent symbol
                return True, (new_row, new_col)

    return False, None


def find_valid_numbers(list_of_lists: list) -> int:
    """
    Find all valid numbers in the list of lists.
    :param list_of_lists:
    :return:
    """
    sum_of_partnumbers = 0
    numbers_with_symbols = []

    # Iterate over the list of lists
    for idl, lst in enumerate(list_of_lists):
        idc = 0
        while idc < len(lst):
            char = lst[idc]
            if char.isdigit():
                # Identify the full number
                number = char
                number_end_index = idc
                # Check if the next character is a digit to get the full number
                while number_end_index + 1 < len(lst) and lst[number_end_index + 1].isdigit():
                    number_end_index += 1
                    number += lst[number_end_index]

                # Check if the number is adjacent to a symbol
                for idx in range(idc, number_end_index + 1):
                    isIt, coords = is_adjacent_to_symbol(list_of_lists, idl, idx)
                    if isIt:
                        numbers_with_symbols.append((number, coords))
                        break

                # Update idc to skip the processed part of the number
                idc = number_end_index + 1
            else:
                idc += 1

    # Sort the list of numbers with symbols by the coordinates of the symbols
    numbers_with_symbols.sort(key=lambda x: x[1])

    # Iterate over the list of numbers with symbols
    for i in range(len(numbers_with_symbols) - 1):
        # Check if the symbols are the same for the current and the next number
        if numbers_with_symbols[i][1] == numbers_with_symbols[i + 1][1]:
            # Add the product of the two numbers to the sum
            gear_ratio = int(int(numbers_with_symbols[i][0]) * int(numbers_with_symbols[i + 1][0]))
            print(f"Found a gear ratio of {gear_ratio}, numbers were {int(numbers_with_symbols[i][0])}, {int(numbers_with_symbols[i + 1][0])}")
            sum_of_partnumbers += int(int(numbers_with_symbols[i][0]) * int(numbers_with_symbols[i + 1][0]))

    return sum_of_partnumbers



# Get the list of strings from the file
list_of_strings = file_to_list("adventofcode.com_2023_day_3_input.txt")

# Convert the list of strings to a list of lists
list_of_lists = list_of_strings_to_list_of_lists(list_of_strings)

# Find the valid numbers
valid_numbers = find_valid_numbers(list_of_lists)

# Print the result
print(f"The sum of the valid numbers is {valid_numbers}.")



In [9]:
2083412
2161828 -> 71407702 # too low
           73698274

           73646890
           

'.'

In [133]:
print(len(np.where(content_array == '*')[0]))


372